In [ ]:
!pip install sqlalchemy
#!pip install pyhive
#!pip install thrift
#!pip install sasl
#!pip install thrift-sasl 

import sqlalchemy
import pandas as pd
from sqlalchemy.engine import create_engine


In [ ]:

engine = create_engine('hive://spark-thrift:10000/default')
pd.read_sql_query("create database if not exists sample",con=engine)
pd.read_sql_query("show databases",con=engine)


In [ ]:
pd.read_sql_query("use sample", con=engine)
pd.read_sql_query("create table if not exists tbl1 (ckey int, cval varchar(64))", con=engine)
pd.read_sql_query("show tables", con=engine)


In [ ]:
pd.read_sql_query("insert into sample.tbl1 values (1, 'a');", con=engine)
pd.read_sql_query("select * from sample.tbl1;", con=engine)

In [ ]:

pd.read_sql_query("show tables", con=engine)

In [ ]:
pd.read_sql_query("""
    create table sample.hudi_tbl (
      id int,
      name string,
      price double,
      ts bigint
    ) using hudi
    tblproperties (
      type = 'cow',
      primaryKey = 'id'
    )
""", con=engine)

In [ ]:
pd.read_sql_query("show tables", con=engine)